# Simplex (piecewise-linear) method on P0 case — Pyomo + Gurobi (default 5 iterations)

Per scenario \(\omega\):

1. Start with endpoints \(y=0,3\).
2. Build piecewise-linear interpolant \(A^s_\omega\) through current sample points.
3. Compute \(m^s_\omega = \min_{y\in[0,3]}(Q_\omega(y)-A^s_\omega(y))\), then shift \(A^s_\omega+m^s_\omega\).
4. Add an argmin point (grid-based) to the interpolation set (scenario-wise points may differ).
5. Repeat for **K=5** iterations.

We compute the min on a dense grid (step \(h=10^{-3}\)). The value \(m^s_\omega\) is obtained via a tiny LP in Pyomo solved by **Gurobi** if available.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pyomo.environ as pyo

# --------------------------
# Match plotting style from your CZ/LG notebook
# --------------------------
plt.rcParams.update({
    "font.family": "serif",
    "mathtext.fontset": "cm",
    "axes.labelsize": 14,
    "xtick.labelsize": 12,
    "ytick.labelsize": 12,
    "legend.fontsize": 12,
    "axes.linewidth": 1.0,
})

FIGSIZE_PANEL = (5.1, 3.6)   # approx size of a single panel (a)/(b) in CZ/LG
FIGSIZE_WIDE  = (10.2, 3.6)  # approx size of wide bottom panel (c) in CZ/LG

Y_L, Y_U = 0.0, 3.0
h = 0.001
ys = np.arange(Y_L, Y_U + 1e-12, h)
N = len(ys)
print("Grid points:", N, " step=", h)

def pick_solver():
    for name in ["gurobi", "gurobi_direct", "gurobi_persistent", "cbc", "glpk", "highs"]:
        try:
            opt = pyo.SolverFactory(name)
            if opt is not None and opt.available(exception_flag=False):
                print("Using solver:", name)
                return opt, name
        except Exception:
            continue
    raise RuntimeError("No LP/MIP solver found. Install Gurobi (recommended) or GLPK/CBC/HiGHS.")

opt, solver_name = pick_solver()


In [ ]:
# --------------------------
# P0 scenario value functions (Appendix C, P0-V)
# --------------------------
def Q1(y):
    y = np.asarray(y)
    return 1.00694*y**3 - 4.74589*y**2 + 5.17523*y

def Q2(y):
    y = np.asarray(y)
    return -0.677232*y**3 + 3.03949*y**2 - 3.02338*y

def Qtot(y):
    return Q1(y) + Q2(y)

Q1_vals = Q1(ys)
Q2_vals = Q2(ys)
Qtot_vals = Q1_vals + Q2_vals


In [ ]:
def piecewise_linear_interp(x_pts, y_pts, x_query):
    # Piecewise-linear interpolant through (x_pts, y_pts), evaluated at x_query. x_pts sorted.
    return np.interp(x_query, x_pts, y_pts)

def compute_ms_via_lp(diff_vals, solver):
    # Compute ms = min_i diff_vals[i] via LP:
    #   maximize ms
    #   s.t. ms <= diff_i  for all i
    m = pyo.ConcreteModel()
    m.ms = pyo.Var(domain=pyo.Reals)
    m.I = pyo.RangeSet(0, len(diff_vals)-1)

    def c_rule(mm, i):
        return mm.ms <= float(diff_vals[i])
    m.c = pyo.Constraint(m.I, rule=c_rule)

    m.obj = pyo.Objective(expr=m.ms, sense=pyo.maximize)
    solver.solve(m, tee=False)
    return float(pyo.value(m.ms))

def simplex_piecewise_underestimator(Q_vals, K=5, y_grid=ys):
    # Per-scenario simplex / piecewise-linear loop on a dense grid.
    x_pts = [float(y_grid[0]), float(y_grid[-1])]  # endpoints 0 and 3
    hist = []

    for k in range(1, K+1):
        x_pts = sorted(set([round(x, 12) for x in x_pts]))
        y_pts = [float(np.interp(x, y_grid, Q_vals)) for x in x_pts]

        As_vals = piecewise_linear_interp(np.array(x_pts), np.array(y_pts), y_grid)
        diff = Q_vals - As_vals  # Q - As

        ms = compute_ms_via_lp(diff, opt)  # equals min on grid
        idx = int(np.argmin(diff))
        y_m = float(y_grid[idx])

        hist.append({"iter": k, "x_pts": x_pts.copy(), "ms": ms, "y_m": y_m, "min_diff": float(diff.min())})

        if all(abs(y_m - x) > 1e-12 for x in x_pts):
            x_pts.append(y_m)

    # final recompute
    x_pts = sorted(set([round(x, 12) for x in x_pts]))
    y_pts = [float(np.interp(x, y_grid, Q_vals)) for x in x_pts]
    As_vals = piecewise_linear_interp(np.array(x_pts), np.array(y_pts), y_grid)
    diff = Q_vals - As_vals
    ms = compute_ms_via_lp(diff, opt)
    As_shift = As_vals + ms
    return x_pts, As_vals, ms, As_shift, hist


In [ ]:
# --------------------------
# Run (default K=5)
# --------------------------
K = 5
x1, As1, ms1, As1_shift, hist1 = simplex_piecewise_underestimator(Q1_vals, K=K)
x2, As2, ms2, As2_shift, hist2 = simplex_piecewise_underestimator(Q2_vals, K=K)

print("Scenario 1: ms1 =", ms1, " final points:", x1)
print("Scenario 2: ms2 =", ms2, " final points:", x2)

sum_shift = As1_shift + As2_shift


In [ ]:
# --------------------------
# Plot helpers (match your style)
# Adjust line widths here:
# --------------------------
LW_TRUE  = 2.0
LW_AS    = 2.0
LW_SHIFT = 2.0

def save_or_show(fig, savepath):
    if savepath:
        fig.savefig(savepath, dpi=300, bbox_inches="tight")
    return fig

def plot_Q_vs_As(Qvals, Asvals, ylabel, color_Q, color_As, label_Q, label_As, savepath=None):
    fig = plt.figure(figsize=FIGSIZE_PANEL)
    ax = fig.add_subplot(111)
    ax.plot(ys, Qvals, color=color_Q, lw=LW_TRUE, label=label_Q)
    ax.plot(ys, Asvals, color=color_As, lw=LW_AS, ls="--", label=label_As)
    ax.set_xlabel(r"$y$")
    ax.set_ylabel(ylabel)
    ax.set_xlim(Y_L, Y_U)
    ax.legend(loc="upper right", frameon=True)
return save_or_show(fig, savepath)

def plot_Q_vs_Shift(Qvals, Shiftvals, ylabel, color_Q, color_Shift, label_Q, label_Shift, savepath=None):
    fig = plt.figure(figsize=FIGSIZE_PANEL)
    ax = fig.add_subplot(111)
    ax.plot(ys, Qvals, color=color_Q, lw=LW_TRUE, label=label_Q)
    ax.plot(ys, Shiftvals, color=color_Shift, lw=LW_SHIFT, ls="--", label=label_Shift)
    ax.set_xlabel(r"$y$")
    ax.set_ylabel(ylabel)
    ax.set_xlim(Y_L, Y_U)
    ax.legend(loc="upper right", frameon=True)
return save_or_show(fig, savepath)

def plot_total(Qtot, sumShift, savepath=None):
    fig = plt.figure(figsize=FIGSIZE_PANEL)
    ax = fig.add_subplot(111)
    ax.plot(ys, Qtot, color="black", lw=LW_TRUE, label=r"$v$")
    ax.plot(ys, sumShift, color="black", lw=LW_SHIFT, ls="--",
            label=r"$(A^s_1+m^s_1) + (A^s_2+m^s_2)$")
    ax.set_xlabel(r"$y$")
    ax.set_ylabel(r"$v(y)$")
    ax.set_xlim(Y_L, Y_U)
    ax.legend(loc="upper right", frameon=True)
return save_or_show(fig, savepath)


In [ ]:
# --------------------------
# Output 5 figures (as requested)
# --------------------------

# 1) Scenario 1: As and true
plot_Q_vs_As(
    Q1_vals, As1,
    ylabel=r"$v_1(y)$",
    color_Q="red", color_As="black",
    label_Q=r"$v_1$", label_As=r"$A^s_1$",
    savepath="simplex_fig1_s1_As.png"
)
plt.show()

# 2) Scenario 2: As and true
plot_Q_vs_As(
    Q2_vals, As2,
    ylabel=r"$v_2(y)$",
    color_Q="deepskyblue", color_As="black",
    label_Q=r"$v_2$", label_As=r"$A^s_2$",
    savepath="simplex_fig2_s2_As.png"
)
plt.show()

# 3) Scenario 1: As+ms and true
plot_Q_vs_Shift(
    Q1_vals, As1_shift,
    ylabel=r"$v_1(y)$",
    color_Q="red", color_Shift="black",
    label_Q=r"$v_1$", label_Shift=r"$A^s_1+m^s_1$",
    savepath="simplex_fig3_s1_As_ms.png"
)
plt.show()

# 4) Scenario 2: As+ms and true
plot_Q_vs_Shift(
    Q2_vals, As2_shift,
    ylabel=r"$v_2(y)$",
    color_Q="deepskyblue", color_Shift="black",
    label_Q=r"$v_2$", label_Shift=r"$A^s_2+m^s_2$",
    savepath="simplex_fig4_s2_As_ms.png"
)
plt.show()

# 5) Total: true v and sum of (As+ms)
plot_total(
    Qtot_vals, sum_shift,
    savepath="simplex_fig5_total.png"
)
plt.show()

print("Saved PNGs:")
print("  simplex_fig1_s1_As.png")
print("  simplex_fig2_s2_As.png")
print("  simplex_fig3_s1_As_ms.png")
print("  simplex_fig4_s2_As_ms.png")
print("  simplex_fig5_total.png")
